<a href="https://colab.research.google.com/github/agam844/Active-Learning/blob/main/Ciphar_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import random

Entire Dataset

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')

model.fit(x_train, y_train, batch_size=64, epochs=100, validation_data=(x_test, y_test), callbacks=[early_stopping])

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Active Learning Low Confidence incorrect

In [4]:
num_entries = min(5000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()
remaining_x_train1 = x_train1[remaining_indices]
remaining_y_train1 = y_train1[remaining_indices]

In [7]:
num_iterations = 10
for iteration in range(num_iterations):

  pred_list = []
  pred_dict = {}
  pred_dict1 = {}

  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
  model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=30, validation_data=(remaining_x_train, remaining_y_train), callbacks=[early_stopping])

  loss, accuracy = model.evaluate(remaining_x_train, remaining_y_train)
  print(f'Test loss: {loss:.4f}')
  print(f'Test accuracy: {accuracy:.4f}')

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 2000

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the class labels with the highest probability
  predictions_int = np.argmax(probabilities, axis=1)

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the highest probability for each sample
  max_probabilities = np.max(probabilities, axis=1)

  for i in range(len(predictions_int)):
    if predictions_int[i] != remaining_y_train1[i]:
      pred_list.append(i)
      pred_dict.update({max_probabilities[i]:i})
    else:
      pred_dict1.update({max_probabilities[i]:i})

  sorted_pred = dict(sorted(pred_dict.items()))
  sorted_pred1 = dict(sorted(pred_dict1.items()))

  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  num_additional_samples = 2000

  values = [value for value in sorted_pred.values()]
  values1 = [value for value in sorted_pred1.values()]
  # print(values)
  # print(values)

  if(len(values)!= num_additional_samples):
    values= values + values1

  values = np.array(list(values))

  # additional_indices = np.argsort(np.max(max_probabilities))[:num_additional_samples]
  additional_indices = values[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)
  remaining_y_train1 = np.delete(remaining_y_train1, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

Epoch 1/3
79/79 [==============================] - 7s 63ms/step - loss: 2.2362 - accuracy: 0.1572 - val_loss: 2.0430 - val_accuracy: 0.2774
Epoch 2/3
79/79 [==============================] - 2s 27ms/step - loss: 1.9764 - accuracy: 0.2780 - val_loss: 1.8033 - val_accuracy: 0.3780
Epoch 3/3
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6957 - accuracy: 0.4092
Test loss: 1.6957
Test accuracy: 0.4092
1407/1407 [==============================] - 3s 2ms/step
[12915, 10856, 32631, 22792, 29462, 17757, 10936, 12020, 12303, 20525, 60, 23266, 27894, 10977, 37399, 3458, 39482, 15619, 17714, 40183, 8001, 12194, 37119, 3031, 40617, 25207, 24897, 28258, 33232, 10530, 42820, 19055, 1034, 8359, 6252, 10668, 32330, 38893, 28815, 1047, 1030, 3556, 36442, 1751, 657, 23791, 12858, 17955, 38190, 11144, 10731, 374, 7652, 42722, 4288, 356, 13357, 22266, 23094, 35652, 1182, 4557, 31358, 42710, 22581, 31673, 8749, 41762, 5740, 4274, 23626, 41381, 6231, 21064, 41248, 29064, 5576, 39202, 193

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

High Confidence, Wrong Labels

In [ ]:
num_entries = min(5000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()
remaining_x_train1 = x_train1[remaining_indices]
remaining_y_train1 = y_train1[remaining_indices]

In [ ]:
num_iterations = 10
for iteration in range(num_iterations):

  pred_list = []
  pred_dict = {}
  pred_dict1 = {}

  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
  model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=30, validation_data=(remaining_x_train, remaining_y_train), callbacks=[early_stopping])

  loss, accuracy = model.evaluate(remaining_x_train, remaining_y_train)
  print(f'Test loss: {loss:.4f}')
  print(f'Test accuracy: {accuracy:.4f}')

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 2000

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the class labels with the highest probability
  predictions_int = np.argmax(probabilities, axis=1)

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the highest probability for each sample
  max_probabilities = np.max(probabilities, axis=1)

  for i in range(len(predictions_int)):
    if predictions_int[i] != remaining_y_train1[i]:
      pred_list.append(i)
      pred_dict.update({max_probabilities[i]:i})
    else:
      pred_dict1.update({max_probabilities[i]:i})

  sorted_pred = dict(sorted(pred_dict.items()))
  sorted_pred1 = dict(sorted(pred_dict1.items()))

  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  num_additional_samples = 2000

  values = [value for value in sorted_pred.values()]
  values1 = [value for value in sorted_pred1.values()]
  # print(values)
  # print(values)

  if(len(values)!= num_additional_samples):
    values= values + values1

  values = np.array(list(values))

  # additional_indices = np.argsort(np.max(max_probabilities))[:num_additional_samples]
  additional_indices = values[-num_additional_samples:]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)
  remaining_y_train1 = np.delete(remaining_y_train1, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Low Confidence, Correct Labels

In [ ]:
num_entries = min(5000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()
remaining_x_train1 = x_train1[remaining_indices]
remaining_y_train1 = y_train1[remaining_indices]

In [ ]:
num_iterations = 10
for iteration in range(num_iterations):

  pred_list = []
  pred_dict = {}
  pred_dict1 = {}

  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
  model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=30, validation_data=(remaining_x_train, remaining_y_train), callbacks=[early_stopping])

  loss, accuracy = model.evaluate(remaining_x_train, remaining_y_train)
  print(f'Test loss: {loss:.4f}')
  print(f'Test accuracy: {accuracy:.4f}')

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 2000

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the class labels with the highest probability
  predictions_int = np.argmax(probabilities, axis=1)

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the highest probability for each sample
  max_probabilities = np.max(probabilities, axis=1)

  for i in range(len(predictions_int)):
    if predictions_int[i] == remaining_y_train1[i]:
      pred_list.append(i)
      pred_dict.update({max_probabilities[i]:i})
    else:
      pred_dict1.update({max_probabilities[i]:i})

  sorted_pred = dict(sorted(pred_dict.items()))
  sorted_pred1 = dict(sorted(pred_dict1.items()))

  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  num_additional_samples = 2000

  values = [value for value in sorted_pred.values()]
  values1 = [value for value in sorted_pred1.values()]
  # print(values)
  # print(values)

  if(len(values)!= num_additional_samples):
    values= values + values1

  values = np.array(list(values))

  # additional_indices = np.argsort(np.max(max_probabilities))[:num_additional_samples]
  additional_indices = values[:num_additional_samples]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)
  remaining_y_train1 = np.delete(remaining_y_train1, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

High Cofidence, Correct labels

In [ ]:
num_entries = min(5000, len(y_train))
max_index = len(x_train)
random_indices = np.random.choice(max_index, size=num_entries, replace=False)

# Select random entries
y_train = np.array(y_train)
selected_x_train = x_train[random_indices]
selected_y_train = y_train[random_indices]

remaining_indices = np.setdiff1d(np.arange(max_index), random_indices)
remaining_x_train = x_train[remaining_indices]
remaining_y_train = y_train[remaining_indices]

(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()
remaining_x_train1 = x_train1[remaining_indices]
remaining_y_train1 = y_train1[remaining_indices]

In [ ]:
num_iterations = 10
for iteration in range(num_iterations):

  pred_list = []
  pred_dict = {}
  pred_dict1 = {}

  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
  model.fit(selected_x_train, selected_y_train, batch_size=64, epochs=30, validation_data=(remaining_x_train, remaining_y_train), callbacks=[early_stopping])

  loss, accuracy = model.evaluate(remaining_x_train, remaining_y_train)
  print(f'Test loss: {loss:.4f}')
  print(f'Test accuracy: {accuracy:.4f}')

  #Select additional samples from remaining entries for labeling
  num_additional_samples = 2000

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the class labels with the highest probability
  predictions_int = np.argmax(probabilities, axis=1)

  # Get predicted probabilities for test data
  probabilities = model.predict(remaining_x_train)

  # Get the highest probability for each sample
  max_probabilities = np.max(probabilities, axis=1)

  for i in range(len(predictions_int)):
    if predictions_int[i] == remaining_y_train1[i]:
      pred_list.append(i)
      pred_dict.update({max_probabilities[i]:i})
    else:
      pred_dict1.update({max_probabilities[i]:i})

  sorted_pred = dict(sorted(pred_dict.items()))
  sorted_pred1 = dict(sorted(pred_dict1.items()))

  #additional_indices = np.argsort(np.max(predictions, axis=1))[-num_additional_samples:]
  num_additional_samples = 2000

  values = [value for value in sorted_pred.values()]
  values1 = [value for value in sorted_pred1.values()]
  # print(values)
  # print(values)

  if(len(values)!= num_additional_samples):
    values= values + values1

  values = np.array(list(values))

  # additional_indices = np.argsort(np.max(max_probabilities))[:num_additional_samples]
  additional_indices = values[-num_additional_samples:]

  # Add the additional samples to the selected dataset
  selected_x_train = np.concatenate((selected_x_train, remaining_x_train[additional_indices]))
  selected_y_train = np.concatenate((selected_y_train, remaining_y_train[additional_indices]))

  # Remove the additional samples from the remaining dataset
  remaining_x_train = np.delete(remaining_x_train, additional_indices, axis=0)
  remaining_y_train = np.delete(remaining_y_train, additional_indices, axis=0)
  remaining_y_train1 = np.delete(remaining_y_train1, additional_indices, axis=0)

  print("Iteration:", iteration + 1)
  print("Number of labeled samples:", len(selected_x_train))
  print("Number of remaining samples:", len(remaining_x_train))

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')